In [1]:
import tensorflow as tf # tensorflow 2.10
import numpy as np
import matplotlib.pyplot as plt
from load_data2 import evals_parser, evals_square_parser, evecs_parser

2022-11-03 10:28:33.030009: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-03 10:28:33.243986: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-03 10:28:33.244020: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-03 10:28:33.302946: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-03 10:28:34.757570: W tensorflow/stream_executor/platform/de

In [2]:
# Importing the datasets
batch_size = 16
tfile = "/home/gaspard/ETHZ/Master_Arbeit/acoupipe_datasets/training_1-100000_csmtriu_1src_he4.0625-1393.4375Hz_ds1-v001_13-Sep-2022.tfrecord"
"""
dataset = tf.data.TFRecordDataset(filenames=[tfile])

evals_dataset = dataset.map(evals_parser)
evals_dataset = evals_dataset.shuffle(buffer_size=10).batch(batch_size)

evals_square_dataset = dataset.map(evals_square_parser)
evals_square_dataset = evals_square_dataset.shuffle(buffer_size=10).batch(batch_size)

evecs_dataset = dataset.map(evecs_parser)
evecs_dataset = evecs_dataset.shuffle(buffer_size=10).batch(batch_size)
"""

'\ndataset = tf.data.TFRecordDataset(filenames=[tfile])\n\nevals_dataset = dataset.map(evals_parser)\nevals_dataset = evals_dataset.shuffle(buffer_size=10).batch(batch_size)\n\nevals_square_dataset = dataset.map(evals_square_parser)\nevals_square_dataset = evals_square_dataset.shuffle(buffer_size=10).batch(batch_size)\n\nevecs_dataset = dataset.map(evecs_parser)\nevecs_dataset = evecs_dataset.shuffle(buffer_size=10).batch(batch_size)\n'

### WGANGP to generate eigenvalues

In [3]:
from evals_WGANGP2 import evals_WGANGP

# Creating GAN
evals_wgangp = evals_WGANGP()

ImportError: Numba needs NumPy 1.21 or less

In [ ]:
evals_wgangp.train(n_epoch=3)

In [4]:
import numpy as np
import matplotlib.pyplot as plt

is_real, sample = evals_wgangp.generate_evals()
sample = np.sort(np.array(sample[0, :, :, 0]).flatten()) # convert sample from (1,8,8,1) to (64,)
sample_dB = 10*np.log10(sample)


if is_real:
    plt.title("generated sample, detected as real")
    plt.plot(sample_dB)
else:
    plt.title("generated sample, detected as fake")
    plt.plot(sample_dB)    

NameError: name 'evals_wgangp' is not defined

### WGANGP to generate eigenvectors

In [ ]:
from evecs_WGANGP2 import evecs_WGANGP

# Creating GAN
evecs_wgangp = evecs_WGANGP()

In [ ]:
evecs_wgangp.train(n_epoch=2)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

is_real, sample = evecs_wgangp.generate_evecs()
print(sample.shape)

sample_real = sample[0, :, :, 0] # get real part from sample
sample_imag = sample[0, :, :, 1] # get imaginary part from sample

if is_real:

    fig = plt.figure(figsize=(15,5))

    plt.subplot(1,2,1)
    plt.title("Real part")
    plt.imshow(sample_real)

    plt.subplot(1,2,2)
    plt.title("Imaginary part")
    plt.imshow(sample_imag)

    fig.suptitle('Sample detect as real', fontsize=16)

    
else:
    fig = plt.figure(figsize=(15,5))

    plt.subplot(1,2,1)
    plt.title("Real part")
    plt.imshow(sample_real)

    plt.subplot(1,2,2)
    plt.title("Imaginary part")
    plt.imshow(sample_imag)

    fig.suptitle('Sample detect as fake', fontsize=16)


### Check validity of obtained data

- Use both WGAN-GPs to generate eigenvalues and eigenvectors
- Use the eigendecomposition formula to recreate (scaled) CSM
- Use obtained CSM to perform beamforming

In [ ]:
from beamforming import beamform, get_csm

evals_are_real, generated_evals = evals_wgangp.generate_evals()
evecs_are_real, generated_evecs = evecs_wgangp.generate_evecs()

#if evals_are_real and evecs_are_real:
    
# reshape data
evals_vec_numpy = np.sort(np.array(generated_evals[0, :, :, 0]).flatten())
evals_vec = tf.convert_to_tensor(evals_vec_numpy)
evecs_real = generated_evecs[0, :, :, 0] # get real part from sample
evecs_imag = generated_evecs[0, :, :, 1] # get imaginary part from sample

print("-=-=-=-=-=-=-=-=-=-=-=-=-")
print(f"real_evecs_real.shape={evecs_real.shape}")
print(f"real_evecs_imag.shape={evecs_imag.shape}")
print(f"tmp_evals_vec.shape={evals_vec.shape}")
print("-=-=-=-=-=-=-=-=-=-=-=-=-")


csm = get_csm(evecs_real, evecs_imag, evals_vec).numpy()

#print(dir(csm))

NUMFREQ = 1 # one freq with index bin = 13 ? -> is that accurate ?
NUMCHANNELS = 64
    

csm = np.reshape(csm, newshape=(NUMFREQ,NUMCHANNELS,NUMCHANNELS))

beamform(csm)



### Data Augmentation 

In [ ]:
from data_augmentation import get_augmentented_dataset

get_augmentented_dataset(n_epoch=1245, size_augmentation=3, size_original_datasets=10)

### GRIDSEARCH: GAN to generate eigenvalues

In [ ]:
from tensorflow import summary
from evals_GAN2 import evals_GAN

lr_list = [5e-7, 1e-7, 5e-8, 1e-8, 5e-9, 1e-9]
#lr_list = [1e-10, 1e-11, 1e-12]
beta_1_list = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]

for lr in lr_list:
    for beta_1 in beta_1_list:
        
        print(f"lr={lr}, beta_1={beta_1}")
        
        evals_gan = evals_GAN(lr, beta_1, summary=False) # Creating GAN
        evals_gan.train(evals_dataset=evals_dataset, n_epoch=500, print_epoch=False) # Training the GAN

        # Plotting sampled eigenvalues 
        is_real, sample = evals_gan.get_evals()
        
        fig = plt.figure()

        if is_real:
            plt.title(f"generated sample, detected as real, lr={lr}, b={beta_1}")
            plt.plot(np.sort(np.array(sample)[0]))
        else:
            plt.title(f"generated sample, detected as fake, lr={lr}, b={beta_1}")
            plt.plot(np.sort(np.array(sample)[0]))

In [ ]:
# Grid search 2
#from tensorflow import summary
from evals_GAN2 import evals_GAN

lr_list = [1e-9, 1e-9, 1e-8, 1e-7, 1e-7]
beta_1_list = [0.5, 0.7, 0.2, 0.1, 0.9]

n_epoch=250

for lr, beta_1 in zip(lr_list, beta_1_list):
    
    print(f"lr={lr}, beta_1={beta_1}")
    
    evals_gan = evals_GAN(lr, beta_1, summary=False) # Creating GAN
    evals_gan.train(evals_dataset=evals_dataset, n_epoch=n_epoch, print_epoch=False) # Training the GAN

    # Plotting sampled eigenvalues 
    is_real, sample = evals_gan.get_evals()
    
    fig = plt.figure()

    if is_real:
        plt.title(f"generated sample, detected as real, lr={lr}, b={beta_1}")
        plt.plot(np.sort(np.array(sample)[0]))
    else:
        plt.title(f"generated sample, detected as fake, lr={lr}, b={beta_1}")
        plt.plot(np.sort(np.array(sample)[0]))
    
# runtime   : 1000 epochs  : approx 5min30
# runtime   : 250 epochs   : approx 1min30

### GAN to generate eigenvalues

In [ ]:
from evals_GAN2 import evals_GAN

# Creating GAN
evals_gan = evals_GAN()

# Training the GAN
evals_gan.train(evals_dataset=evals_dataset, n_epoch=100)

In [ ]:
# Plotting sampled eigenvalues 
is_real, sample = evals_gan.get_evals()

if is_real:
    plt.title("generated sample, detected as real")
    plt.plot(np.sort(np.array(sample)[0]))
else:
    plt.title("generated sample, detected as fake")
    plt.plot(np.sort(np.array(sample)[0]))

### DCGAN to generate eigenvalues

In [ ]:
from evals_DCGAN2 import evals_DCGAN

# Creating GAN
evals_dcgan = evals_DCGAN()

# Training the GAN
evals_dcgan.train(evals_dataset=evals_dataset, n_epoch=300)

In [ ]:
# Plotting sampled eigenvalues 
is_real, sample = evals_dcgan.get_evals()

if is_real:
    plt.title("generated sample, detected as real")
    plt.plot(np.sort(np.array(sample)[0]))
else:
    plt.title("generated sample, detected as fake")
    plt.plot(np.sort(np.array(sample)[0]))

### WGAN to generate eigenvectors 

In [ ]:
from evecs_WGAN2 import evecs_WGAN

# Creating WGAN
evecs_wgan = evecs_WGAN()

In [ ]:
evecs_wgan.train(evecs_dataset=evecs_dataset, n_epoch=1000)

In [ ]:
sample = evecs_wgan.get_evecs()